Install Faker

In [0]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.9/1.9 MB 61.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/347.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 24.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

Generate synthetic dataset with a few PCI / PII attributes

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from faker import Faker
import random
import string

# Initialize Spark Session
spark = SparkSession.builder.appName("PII_PCI_Dataset").getOrCreate()

fake = Faker()

# Function to generate valid credit card details
def generate_credit_card():
    card_number = fake.credit_card_number(card_type=None)  # Generates a random credit card number
    expiry_date = fake.credit_card_expire()  # Random expiration date
    cvv = fake.credit_card_security_code()  # Random CVV
    return card_number, expiry_date, cvv


# Function to generate a fake PAN number (format: ABCDE1234F)
# def generate_pan_number():
#     letters = ''.join(random.choices(string.ascii_uppercase, k=5))  # First 5 letters
#     digits = ''.join(random.choices(string.digits, k=4))  # Next 4 digits
#     last_letter = random.choice(string.ascii_uppercase)  # Last letter
#     return f"{letters}{digits}{last_letter}"

# Function to generate a fake PAN number (format: ABCDE1234F)
def generate_pan_number():
    digits = ''.join(random.choices(string.digits, k=16))  
    return f"{digits}"

# Generate sample data
data = []
for _ in range(1000):  # Adjust number of records as needed
    card_number, expiry_date, cvv = generate_credit_card()
    pan_number = generate_pan_number()
    data.append((
        fake.name(),
        fake.email(),
        fake.phone_number(),
        fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%Y-%m-%d"),
        fake.ssn(),
        pan_number,
        fake.address(),
        #card_number,
        expiry_date,
        #cvv,
        fake.company() + " Bank"
    ))

# Define Schema for Spark DataFrame
schema = StructType([
    StructField("full_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone_number", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("ssn", StringType(), True),
    StructField("pan", StringType(), True),
    StructField("address", StringType(), True),
    #StructField("ccn", StringType(), True),
    StructField("exp_date", StringType(), True),
    #StructField("cvv", StringType(), True),
    StructField("bank_name", StringType(), True),
])

# Create Spark DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show Sample Data
#df.show(truncate=False)
#display(df)

Create and load the snthetic data to a table in landing catalog

In [0]:
%sql
create or replace table envlp_encryp_landing.pci_pii_dataset.customer_records_1 (
  full_name string,
  email string,
  phone_number string,
  dob string,
  ssn string,
  pan string,
  address string,
  exp_date string,
  bank_name string
  );

In [0]:
df.write.format("delta")\
    .mode("append")\
    .saveAsTable("envlp_encryp_landing.pci_pii_dataset.customer_records_1")